In [1]:
%matplotlib notebook

import multiprocessing
import numpy as np

import sklearn.model_selection

from recova.clustering import IdentityClusteringAlgorithm
from recova.covariance import CensiCovarianceComputationAlgorithm, SamplingCovarianceComputationAlgorithm
from recova.registration_result_database import RegistrationPairDatabase
from recova.util import kullback_leibler, bat_distance

# Split into training and validation dataset

In [2]:
db = RegistrationPairDatabase('/home/dlandry/dataset/normal-estimates/')
pairs = db.registration_pairs()

pairs_training, pairs_validation = sklearn.model_selection.train_test_split(pairs, test_size=0.3)

# Reference covariances of validation

In [3]:
clustering_algo = IdentityClusteringAlgorithm()
reference_algo = SamplingCovarianceComputationAlgorithm(clustering_algo)

with multiprocessing.Pool(7) as pool:
    reference_covariances = pool.map(reference_algo.compute, pairs_validation)

In [4]:
reference_covariances = np.array(reference_covariances)

# Compute Censi covariances

In [5]:
censi_algo = CensiCovarianceComputationAlgorithm()

with multiprocessing.Pool(7) as pool:
    censi_covariances = pool.map(censi_algo.compute, pairs_validation)



censi_estimate -reading /home/dlandry/dataset/normal-estimates/hauptgebaude-22-21/reading.pcd -reference /home/dlandry/dataset/normal-estimates/hauptgebaude-22-21/reference.pcd -ground_truth '[[0.9993835465584582, 0.034589299695208034, 0.005804517967634852, 0.7332930716397765], [-0.03458462375581589, 0.9994004408640366, -0.001063007347316315, 0.007106754931644144], [-0.005837992457469581, 0.0008616924849857932, 0.9999816150373283, 0.01510346688920472], [0.0, 0.0, 0.0, 1.0]]' -config '{errorMinimizer: PointToPlaneWithCovErrorMinimizer, inspector: NullInspector, logger: NullLogger,
  matcher: {KDTreeMatcher: {knn: 3}}, outlierFilters: [{TrimmedDistOutlierFilter: {
        ratio: 0.7}}], readingDataPointsFilters: [{SurfaceNormalDataPointsFilter: {
        keepDensities: 1, knn: 20}}, {MaxDensityDataPointsFilter: {maxDensity: 50000}},
    {RandomSamplingDataPointsFilter: {prob: 0.75}}], referenceDataPointsFilters: [
    {SurfaceNormalDataPointsFilter: {keepDensities: 1, knn: 20}}, {MaxDens

'
censi_estimate -reading /home/dlandry/dataset/normal-estimates/wood_autumn-24-22/reading.pcd -reference /home/dlandry/dataset/normal-estimates/wood_autumn-24-22/reference.pcd -ground_truth '[[0.9914780116842357, 0.12976412991977612, 0.011602837237993557, 1.3961936632155663], [-0.1293781969027613, 0.9911591185606495, -0.029420471643185283, -0.010743835341847863], [-0.015317606882827274, 0.027669471159987732, 0.999500057056272, 0.003087008643660072], [0.0, 0.0, 0.0, 1.0]]' -config '{errorMinimizer: PointToPlaneWithCovErrorMinimizer, inspector: NullInspector, logger: NullLogger,
  matcher: {KDTreeMatcher: {knn: 3}}, outlierFilters: [{TrimmedDistOutlierFilter: {
        ratio: 0.7}}], readingDataPointsFilters: [{SurfaceNormalDataPointsFilter: {
        keepDensities: 1, knn: 20}}, {MaxDensityDataPointsFilter: {maxDensity: 50000}},
    {RandomSamplingDataPointsFilter: {prob: 0.75}}], referenceDataPointsFilters: [
    {SurfaceNormalDataPointsFilter: {keepDensities: 1, knn: 20}}, {MaxDensit

'
censi_estimate -reading /home/dlandry/dataset/normal-estimates/wood_autumn-31-28/reading.pcd -reference /home/dlandry/dataset/normal-estimates/wood_autumn-31-28/reference.pcd -ground_truth '[[0.994620449997449, 0.10347153681979972, -0.00473477617835592, 2.045337496051225], [-0.1034344371527015, 0.9946087114817166, 0.0074453275117813775, -0.1076993771932111], [0.005478738601685579, -0.006915563868521496, 0.999961425297291, 0.0069026041588591824], [0.0, 0.0, 0.0, 1.0]]' -config '{errorMinimizer: PointToPlaneWithCovErrorMinimizer, inspector: NullInspector, logger: NullLogger,
  matcher: {KDTreeMatcher: {knn: 3}}, outlierFilters: [{TrimmedDistOutlierFilter: {
        ratio: 0.7}}], readingDataPointsFilters: [{SurfaceNormalDataPointsFilter: {
        keepDensities: 1, knn: 20}}, {MaxDensityDataPointsFilter: {maxDensity: 50000}},
    {RandomSamplingDataPointsFilter: {prob: 0.75}}], referenceDataPointsFilters: [
    {SurfaceNormalDataPointsFilter: {keepDensities: 1, knn: 20}}, {MaxDensityDa

'
censi_estimate -reading /home/dlandry/dataset/normal-estimates/wood_autumn-04-01/reading.pcd -reference /home/dlandry/dataset/normal-estimates/wood_autumn-04-01/reference.pcd -ground_truth '[[0.8512681879987252, 0.5232721478760674, -0.039073400500076266, 1.5167784397067885], [-0.5232017388322671, 0.8521126087073128, 0.01283788394355523, 0.18949147167765218], [0.04001233370120083, 0.009514682210002177, 0.9991535101334579, 0.08820825063452552], [0.0, 0.0, 0.0, 1.0]]' -config '{errorMinimizer: PointToPlaneWithCovErrorMinimizer, inspector: NullInspector, logger: NullLogger,
  matcher: {KDTreeMatcher: {knn: 3}}, outlierFilters: [{TrimmedDistOutlierFilter: {
        ratio: 0.7}}], readingDataPointsFilters: [{SurfaceNormalDataPointsFilter: {
        keepDensities: 1, knn: 20}}, {MaxDensityDataPointsFilter: {maxDensity: 50000}},
    {RandomSamplingDataPointsFilter: {prob: 0.75}}], referenceDataPointsFilters: [
    {SurfaceNormalDataPointsFilter: {keepDensities: 1, knn: 20}}, {MaxDensityDataP

'
censi_estimate -reading /home/dlandry/dataset/normal-estimates/gazebo_summer-30-27/reading.pcd -reference /home/dlandry/dataset/normal-estimates/gazebo_summer-30-27/reference.pcd -ground_truth '[[0.9992663263953641, -0.0072689590885272335, -0.03760452609704082, 1.330138438412458], [0.007313400467765588, 0.9999723415625901, 0.0010703530303321337, 0.18417412281797896], [0.03759438499240926, -0.0013438143594608845, 0.9992905303092613, 0.0922770751279883], [0.0, 0.0, 0.0, 1.0]]' -config '{errorMinimizer: PointToPlaneWithCovErrorMinimizer, inspector: NullInspector, logger: NullLogger,
  matcher: {KDTreeMatcher: {knn: 3}}, outlierFilters: [{TrimmedDistOutlierFilter: {
        ratio: 0.7}}], readingDataPointsFilters: [{SurfaceNormalDataPointsFilter: {
        keepDensities: 1, knn: 20}}, {MaxDensityDataPointsFilter: {maxDensity: 50000}},
    {RandomSamplingDataPointsFilter: {prob: 0.75}}], referenceDataPointsFilters: [
    {SurfaceNormalDataPointsFilter: {keepDensities: 1, knn: 20}}, {MaxDe

...

'
censi_estimate -reading /home/dlandry/dataset/normal-estimates/apartment-33-31/reading.pcd -reference /home/dlandry/dataset/normal-estimates/apartment-33-31/reference.pcd -ground_truth '[[0.8573069838941393, 0.5147987041504609, 0.002770248370393505, 0.9167983600091505], [-0.5147729504676699, 0.8572994381606595, -0.006631629797349224, -0.11404034923927853], [-0.005789499101204847, 0.0042600048242294256, 0.9999750154466681, 0.009272462860213652], [0.0, 0.0, 0.0, 1.0]]' -config '{errorMinimizer: PointToPlaneWithCovErrorMinimizer, inspector: NullInspector, logger: NullLogger,
  matcher: {KDTreeMatcher: {knn: 3}}, outlierFilters: [{TrimmedDistOutlierFilter: {
        ratio: 0.7}}], readingDataPointsFilters: [{SurfaceNormalDataPointsFilter: {
        keepDensities: 1, knn: 20}}, {MaxDensityDataPointsFilter: {maxDensity: 50000}},
    {RandomSamplingDataPointsFilter: {prob: 0.75}}], referenceDataPointsFilters: [
    {SurfaceNormalDataPointsFilter: {keepDensities: 1, knn: 20}}, {MaxDensityDat

'
censi_estimate -reading /home/dlandry/dataset/normal-estimates/hauptgebaude-18-16/reading.pcd -reference /home/dlandry/dataset/normal-estimates/hauptgebaude-18-16/reference.pcd -ground_truth '[[0.9999844332322544, -0.005432806508280771, 0.0009322657841930211, 1.593167357770234], [0.005438386039915064, 0.9999718661108, -0.005185220171384153, -0.009873306086519595], [-0.0009045330863992767, 0.005190260487598022, 0.9999860061084251, 0.038255190163280356], [0.0, 0.0, 0.0, 1.0]]' -config '{errorMinimizer: PointToPlaneWithCovErrorMinimizer, inspector: NullInspector, logger: NullLogger,
  matcher: {KDTreeMatcher: {knn: 3}}, outlierFilters: [{TrimmedDistOutlierFilter: {
        ratio: 0.7}}], readingDataPointsFilters: [{SurfaceNormalDataPointsFilter: {
        keepDensities: 1, knn: 20}}, {MaxDensityDataPointsFilter: {maxDensity: 50000}},
    {RandomSamplingDataPointsFilter: {prob: 0.75}}], referenceDataPointsFilters: [
    {SurfaceNormalDataPointsFilter: {keepDensities: 1, knn: 20}}, {MaxDe

'
censi_estimate -reading /home/dlandry/dataset/normal-estimates/gazebo_summer-06-05/reading.pcd -reference /home/dlandry/dataset/normal-estimates/gazebo_summer-06-05/reference.pcd -ground_truth '[[0.999986047900237, 0.005166860500813241, 0.0016323861011873645, 0.5251826531846047], [-0.005176204693385838, 0.9999692187008886, 0.006060575379877916, 0.06930742913627791], [-0.0016005417941556003, -0.006069231388724261, 0.9999802575147017, 0.004052510949903751], [0.0, 0.0, 0.0, 1.0]]' -config '{errorMinimizer: PointToPlaneWithCovErrorMinimizer, inspector: NullInspector, logger: NullLogger,
  matcher: {KDTreeMatcher: {knn: 3}}, outlierFilters: [{TrimmedDistOutlierFilter: {
        ratio: 0.7}}], readingDataPointsFilters: [{SurfaceNormalDataPointsFilter: {
        keepDensities: 1, knn: 20}}, {MaxDensityDataPointsFilter: {maxDensity: 50000}},
    {RandomSamplingDataPointsFilter: {prob: 0.75}}], referenceDataPointsFilters: [
    {SurfaceNormalDataPointsFilter: {keepDensities: 1, knn: 20}}, {Ma

'
censi_estimate -reading /home/dlandry/dataset/normal-estimates/stairs-28-25/reading.pcd -reference /home/dlandry/dataset/normal-estimates/stairs-28-25/reference.pcd -ground_truth '[[0.7908773977831141, -0.6113482269192231, -0.02765982025132826, 1.3655755140792207], [0.6106385481888501, 0.7853608255773036, 0.10163627239099869, 0.6608297760341033], [-0.04041231049344732, -0.0972713421166397, 0.9944370944935966, 0.20505534879362974], [0.0, 0.0, 0.0, 1.0]]' -config '{errorMinimizer: PointToPlaneWithCovErrorMinimizer, inspector: NullInspector, logger: NullLogger,
  matcher: {KDTreeMatcher: {knn: 3}}, outlierFilters: [{TrimmedDistOutlierFilter: {
        ratio: 0.7}}], readingDataPointsFilters: [{SurfaceNormalDataPointsFilter: {
        keepDensities: 1, knn: 20}}, {MaxDensityDataPointsFilter: {maxDensity: 50000}},
    {RandomSamplingDataPointsFilter: {prob: 0.75}}], referenceDataPointsFilters: [
    {SurfaceNormalDataPointsFilter: {keepDensities: 1, knn: 20}}, {MaxDensityDataPointsFilter:

'
censi_estimate -reading /home/dlandry/dataset/normal-estimates/apartment-31-30/reading.pcd -reference /home/dlandry/dataset/normal-estimates/apartment-31-30/reference.pcd -ground_truth '[[0.9383394454176333, 0.3455117517901022, -0.01177072906782989, 0.5563494963821798], [-0.3456175790431025, 0.9383386036735223, -0.008488846660980919, -0.19663130320400146], [0.008113831912639804, 0.012032311134746565, 0.9998937170271254, 0.009009810607178363], [0.0, 0.0, 0.0, 1.0]]' -config '{errorMinimizer: PointToPlaneWithCovErrorMinimizer, inspector: NullInspector, logger: NullLogger,
  matcher: {KDTreeMatcher: {knn: 3}}, outlierFilters: [{TrimmedDistOutlierFilter: {
        ratio: 0.7}}], readingDataPointsFilters: [{SurfaceNormalDataPointsFilter: {
        keepDensities: 1, knn: 20}}, {MaxDensityDataPointsFilter: {maxDensity: 50000}},
    {RandomSamplingDataPointsFilter: {prob: 0.75}}], referenceDataPointsFilters: [
    {SurfaceNormalDataPointsFilter: {keepDensities: 1, knn: 20}}, {MaxDensityDataP

In [6]:
losses = [kullback_leibler(censi_covariances[i], reference_covariances[i]) for i in range(len(reference_covariances))]
losses2 = [kullback_leibler(reference_covariances[i], censi_covariances[i]) for i in range(len(reference_covariances))]

In [7]:
losses = np.array(losses)
losses2 = np.array(losses2)

losses = losses + losses2

In [13]:
print(np.mean(losses))
print(np.std(losses))

8897230216.932716
5968405127.607062


In [16]:
np.linalg.det(censi_covariances[12])

1.0949221863486257e-56

In [15]:
np.linalg.det(reference_covariances[12])

0.1021633654619693

In [ ]:
bat_distances = [bat_distance(reference_covariances[i], censi_covariances[i]) for i in range(len(reference_covariances))]

In [ ]:
bat_distances